### Configuring credentials
To run the following code, ensure that you have the following environment variables set:
* `SF_USERNAME`
* `SF_PASSWORD`
* `SF_ACCOUNT`
* `DATABASE_NAME`

This example uses data from TPC-H. In your snowflake account, ensure that you can access the [TPC-H sample database](https://docs.snowflake.com/en/user-guide/sample-data-tpch).

In [3]:
import os
username=os.environ["SF_USERNAME"]
password=os.environ["SF_PASSWORD"]
account=os.environ["SF_ACCOUNT"]
warehouse="TEST_WH"
database=os.environ["DATABASE_NAME"]
schema="TPCH_SF10"

## Predicate Pushdown
Bodo does optimization in your code, when you are running a workload against snowflake.
In the below query, user is doing a naive Select * query into snowflake.
He is the filtering the dataframe based on Shipdate and dropping columns that are not needed.
In regular pandas, this would have caused unnecessary i/o and memory usage because all the data from snowflake, would have been brought it.
Bodo on the other hand will automatically detect these and push the filter and the needed columns into the query , hence improving i/o and memory utilization.
Run the cell below and check the query in snowflake history.

In [ ]:
import bodo
import pandas as pd
@bodo.jit(cache=True)
def load_lineitem(schema):
    date = pd.Timestamp("1998-09-02")
    lineitem=pd.read_sql(f"select * from {schema}.LINEITEM", f"snowflake://{username}:{password}@{account}/{database}/PUBLIC?warehouse={warehouse}",)
    lineitem=lineitem[lineitem.l_shipdate <= date]
    lineitem=lineitem[["l_quantity", "l_shipdate"]]
    return lineitem
lineitem=load_lineitem(schema)

In [5]:
lineitem.shape

(59142609, 2)

In [6]:
# Let's clear memory before running another query
del lineitem

## Superfast connector
Bodo also has an improved snowflake connector. Under the hood, the result of the above query is shared across multiple cores in the cluster. Traditional etl tools, will do this sharding by doing an initial range query and then submitting multiple select queries for each of the range. Ex. if we are accessing a table employee, traditional etl tool with run the query with a count(*)
select count(*) from 'query'.
Once the count is established, the count will be divided by parallelism needed, lets say 8 in this case, 8 queries with different ranges will be pushed to snowflake.

This causes unnecessary load on the database and also is not truelly parallel. 
Bodo , will only submit one query to snowflake and the connector will receive sharded data from snowflake. 
Run the code below and check snowflake history, you should see only one query submitted.

In [7]:
import pandas as pd
import bodo
import time
@bodo.jit(cache=True)
def tpch_q01_filter(schema):
    t1 = time.time()
    lineitem=pd.read_sql(f"select * from {schema}.LINEITEM", f"snowflake://{username}:{password}@{account}/{database}/PUBLIC?warehouse={warehouse}",)
    date = pd.Timestamp("1998-09-02")
    sel = lineitem.l_shipdate <= date
    lineitem_filtered = lineitem[["l_quantity", "l_extendedprice", "l_discount", "l_tax", "l_returnflag", "l_linestatus",  "l_shipdate", "l_orderkey"]]
    lineitem_filtered = lineitem_filtered[sel]
    lineitem_filtered["avg_qty"] = lineitem_filtered.l_quantity
    lineitem_filtered["avg_price"] = lineitem_filtered.l_extendedprice
    lineitem_filtered["disc_price"] = lineitem_filtered.l_extendedprice * (1 - lineitem_filtered.l_discount)
    lineitem_filtered["charge"] = (
        lineitem_filtered.l_extendedprice * (1 - lineitem_filtered.l_discount) * (1 + lineitem_filtered.l_tax)
    )
    gb = lineitem_filtered.groupby(["l_returnflag", "l_linestatus"], as_index=False)[
        "l_quantity",
        "l_extendedprice",
        "disc_price",
        "charge",
        "avg_qty",
        "avg_price",
        "l_discount",
        "l_orderkey",
    ]
    total = gb.agg(
        {
            "l_quantity": "sum",
            "l_extendedprice": "sum",
            "disc_price": "sum",
            "charge": "sum",
            "avg_qty": "mean",
            "avg_price": "mean",
            "l_discount": "mean",
            "l_orderkey": "count",
        }
    )
    total = total.sort_values(["l_returnflag", "l_linestatus"])
    print(len(total))
    print("Q01 Execution time (s): ", time.time() - t1)
    return total

q1_result=tpch_q01_filter(schema)

4
Q01 Execution time (s):  62.63550699999996


In [8]:
q1_result

,l_returnflag,l_linestatus,l_quantity,l_extendedprice,disc_price,charge,avg_qty,avg_price,l_discount,l_orderkey
3,A,F,377518399.0,566065727797.259766,537759104278.070068,559276670892.124634,25.500975,38237.151009,0.050007,14804077
1,N,F,9851614.0,14767438399.170021,14028805792.211384,14590490998.366734,25.522448,38257.81066,0.049973,385998
2,N,O,743124873.0,1114302286901.89917,1058580922144.967529,1100937000170.611816,25.498076,38233.902923,0.050001,29144351
0,R,F,377732830.0,566431054975.996826,538110922664.77124,559634780885.087769,25.508385,38251.219274,0.049997,14808183
